<a href="https://colab.research.google.com/github/badro97/NLP_Practice/blob/main/%ED%86%A0%ED%81%AC%EB%82%98%EC%9D%B4%EC%A0%80_%EB%8B%A8%EC%96%B4%EC%82%AC%EC%A0%84%EC%9E%90%EB%8F%99%ED%99%94.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive', force_remount=True)

In [ ]:
!pip install konlpy
!pip install hgtk

토크나이저로 1차 분류 후 단어사전 단어 생성하기

토크나이저는 Kkma 사용

- 대용량 데이터 처리 속도가 가장 빠름

In [ ]:
import hgtk
import numpy as np
import pandas as pd
from tqdm import tqdm
from konlpy.tag import Kkma
from konlpy.tag import Okt
from konlpy.tag import Hannanum

In [ ]:
kkma =Kkma()
okt = Okt()
hannanum = Hannanum()

In [ ]:
def word_to_jamo(token):
    def to_special_token(jamo):
      if not jamo:
        return '-'
      else:
        return jamo
    decomposed_token = ''
    jong_list = []
    for char in token:
        try:
        # char( 음 절 ) 을 초 성 , 중 성 , 종 성 으 로 분 리
            cho, jung, jong = hgtk.letter.decompose(char)

            # 자 모 가 빈 문 자 일 경 우 특 수 문 자 - 로 대 체
            jong = to_special_token(jong)
            jong_list.append(jong)
              
            # decomposed_token = decomposed_token + cho + jung + jong

        # 만 약 char( 음 절 ) 이 한 글 이 아 닐 경 우 자 모 를 나 누 지 않 고 추 가
        except Exception as exception:
            if type(exception).__name__ == 'NotHangulException':
                decomposed_token += char
                
    return jong_list

In [ ]:
text = '해물왕해신짬뽕'

In [ ]:
kkma.morphs(text)

1. 토큰마다 붙여보고 종성 기준 단어 생성

2. 붙이는 건 한 개부터 ~ 하나 씩 늘려가며 전체를 다 붙일 때 까지 진행

In [ ]:
def token_collector(text):
  token = kkma.morphs(text)
  token_list = []
  word_list = []
  print(token)
  for i in token:
    for j in range(token.index(i), len(token)-1):
      i += token[j+1]
      jamo = word_to_jamo(i)
      token_list.append(jamo)
      word_list.append(i)
  return token_list, word_list

In [ ]:
token_list, word_list = token_collector(text)

In [ ]:
token_list

In [ ]:
word_list

In [ ]:
token_list2, word_list2 = token_collector("소주칵테일")

### 종성 구분 규칙

종성 유무로 단어 구분

종성 없음 - 

종성 있음 ㅇ

1. -ㅇ > -ㅇㅇㅇ- > -ㅇㅇㅇ
2. -ㅇ- > -ㅇ-ㅇㅇ- > -ㅇ-ㅇㅇ
3. ㅇㅇ > ㅇㅇ- - ㅇㅇ- > ㅇㅇ- (-ㅇㅇ(1))
4. ㅇ- > ㅇ- -ㅇ > ㅇ- -

In [ ]:
def split_check(title):
  def token_collector(text):
    token = kkma.morphs(text)
    token_list = []
    word_list = []
    for i in token:
      for j in range(token.index(i), len(token)-1):
        i += token[j+1]
        jamo = word_to_jamo(i)
        token_list.append(jamo)
        word_list.append(i)
    return token, token_list, word_list

  token, token_list, word_list = token_collector(title)
  check = []
  for tok, word in zip(token_list, word_list):
    pos = np.where(np.array(tok) == '-')[0]
    pos_len = len(pos)
    # 종성이 하나만 없고 맨 앞이라면
    if pos_len == 1 and pos[0] == 0:
      check.append(word)
    # 종성이 하나만 없다면
    elif pos_len == 1:
      check.append(word[:pos[0]])
      check.append(word[pos[0]:])
    # 종성이 여러개 없다면
    elif pos_len != 1:
      # 아예 종성이 없는 단어라면
      if pos_len == len(token) and len(token) > 3:
        check.append(word)
        
      # elif pos_len > 3:
      #   check.append(word[pos[0]:])
      #   for i in range(pos_len-1):
      #     if pos[i+1] == pos[i]+1:
      #       check.append(word[pos[i-1]:pos[i]])
      #       check.append(word[pos[i]:pos[i+1]])
      #       check.append(word[pos[i+1]:])
            
      #     else:
      #       check.append(word[pos[0]:])
  result = check + token
  result = list(set(result))
  result = [v for v in result if v]
  return result

In [ ]:
split_check('해물왕해신짬뽕')

In [ ]:
split_check('보쌈정식')

In [ ]:
split_check('가마솥밥보쌈정식')

In [ ]:
split_check('후라이드오뎅탕')

In [ ]:
kkma.morphs('후라이드오뎅탕')

In [ ]:
df = pd.read_csv('/content/gdrive/MyDrive/싼데비스탄/토큰수작업관련/back_token_count_1.csv', encoding='UTF-8', index_col=0)

In [ ]:
title = df['title']

In [ ]:
tokens = []
for t in tqdm(title):
  token = split_check(t)
  tokens.append(token)
tokens


## 막힌 부분

종성없음 - 를 기준으로 로 슬라이싱 (종성 구분 규칙 구현)

기존 토크나이저의 형태소 분류 문제

## 결론

수작업하러가자...ㅠ